# Stochastic Gradient Descent

We will use the house price prediction dataset available on kaggle ([House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/)).

I strongly suggest, as additional reading, exploring the notebooks from this competition and learning from the competitors and perhaps even participating in the competition!